In [1]:
from langgraph.graph import StateGraph, START, END
import json
from estado import AgentState
from estado import Nivel
from typing import Dict, Any, List
from langchain_core.tools import tool
import os

import requests

In [3]:
import requests

def _dedupe_urls(urls: List[str]) -> List[str]:
    seen = set()
    out = []
    for u in urls:
        if u and u not in seen:
            seen.add(u)
            out.append(u)
    return out

def _normalize_result(results: List[Dict[str, Any]], max_urls: int = 8) -> Dict[str, Any]:
    """
    results: [{"url": "...", "title": "...", "snippet": "..."}]
    """
    urls = _dedupe_urls([r.get("url") for r in results if r.get("url")])
    return {
        "urls": urls[:max_urls],
        "results": results[:max_urls],
        "count": min(len(urls), max_urls),
    }

# -----------------------------
# Tool 1: Tavily
# -----------------------------
@tool("tavily_search_urls")
def tavily_search_urls(query: str, max_results: int = 8) -> str:
    """
    Busca en Tavily y retorna URLs útiles como JSON string.
    Requiere TAVILY_API_KEY en env.
    """
    try:
        from tavily import TavilyClient
        api_key = os.getenv("TAVILY_API_KEY")
        if not api_key:
            return json.dumps({"error": "Missing TAVILY_API_KEY", "urls": [], "results": [], "count": 0})

        client = TavilyClient(api_key=api_key)
        resp = client.search(
            query=query,
            max_results=max_results,
            include_answer=False,
            include_raw_content=False,
            include_images=False,
        )
        items = resp.get("results", []) or []
        results = [{"url": it.get("url"), "title": it.get("title"), "snippet": it.get("content")} for it in items]
        return json.dumps(_normalize_result(results, max_urls=max_results), ensure_ascii=False)
    except Exception as e:
        return json.dumps({"error": f"Tavily failed: {e}", "urls": [], "results": [], "count": 0})

# -----------------------------
# Tool 2: OpenAI "web search"
# -----------------------------
@tool("openai_websearch_urls")
def openai_websearch_urls(query: str, max_results: int = 8) -> str:
    """
    Hace web search con OpenAI (requiere modelo/feature con web search habilitado).
    Retorna URLs como JSON string.
    """
    try:
        llm = ChatOpenAI(model="gpt-4o-mini")  # ajusta tu modelo
        # LangChain no estandariza 100% la salida de "web search" entre versiones.
        # Patrón robusto: pedirle al modelo SOLO JSON con urls y títulos a partir de búsqueda.
        prompt = f"""
Necesito que uses navegación web para buscar: {query}
Devuélveme SOLO un JSON con este formato exacto:
{{
  "results": [{{"url": "...", "title": "...", "snippet": "..."}}],
  "urls": ["..."]
}}
Incluye máximo {max_results} resultados. No agregues texto adicional.
"""
        resp = llm.invoke(prompt)
        text = resp.content.strip()

        # intenta parsear JSON directo
        data = json.loads(text)
        results = data.get("results", []) or []
        # si no viene urls, construirlas desde results
        if not data.get("urls"):
            data["urls"] = [r.get("url") for r in results if r.get("url")]
        data["urls"] = _dedupe_urls(data["urls"])[:max_results]
        data["results"] = results[:max_results]
        data["count"] = len(data["urls"])
        return json.dumps(data, ensure_ascii=False)

    except Exception as e:
        return json.dumps({"error": f"OpenAI websearch failed: {e}", "urls": [], "results": [], "count": 0})

# -----------------------------
# Tool 3A: Google Custom Search JSON API
# -----------------------------
@tool("google_cse_search_urls")
def google_cse_search_urls(query: str, max_results: int = 8) -> str:
    """
    Google Custom Search JSON API.
    Requiere GOOGLE_API_KEY y GOOGLE_CSE_ID.
    """
    try:
        api_key = os.getenv("GOOGLE_API_KEY")
        cse_id = os.getenv("GOOGLE_CSE_ID")
        if not api_key or not cse_id:
            return json.dumps({"error": "Missing GOOGLE_API_KEY or GOOGLE_CSE_ID", "urls": [], "results": [], "count": 0})

        url = "https://www.googleapis.com/customsearch/v1"
        params = {"key": api_key, "cx": cse_id, "q": query, "num": min(max_results, 10)}
        r = requests.get(url, params=params, timeout=20)
        r.raise_for_status()
        data = r.json()

        items = data.get("items", []) or []
        results = [{"url": it.get("link"), "title": it.get("title"), "snippet": it.get("snippet")} for it in items]
        return json.dumps(_normalize_result(results, max_urls=max_results), ensure_ascii=False)
    except Exception as e:
        return json.dumps({"error": f"Google CSE failed: {e}", "urls": [], "results": [], "count": 0})

# -----------------------------
# Tool 3B (alternativa): SerpAPI
# -----------------------------
@tool("serpapi_search_urls")
def serpapi_search_urls(query: str, max_results: int = 8) -> str:
    """
    Alternativa: SerpAPI.
    Requiere SERPAPI_API_KEY.
    """
    try:
        api_key = os.getenv("SERPAPI_API_KEY")
        if not api_key:
            return json.dumps({"error": "Missing SERPAPI_API_KEY", "urls": [], "results": [], "count": 0})

        url = "https://serpapi.com/search.json"
        params = {"engine": "google", "q": query, "api_key": api_key, "num": max_results}
        r = requests.get(url, params=params, timeout=20)
        r.raise_for_status()
        data = r.json()

        organic = data.get("organic_results", []) or []
        results = [{"url": it.get("link"), "title": it.get("title"), "snippet": it.get("snippet")} for it in organic]
        return json.dumps(_normalize_result(results, max_urls=max_results), ensure_ascii=False)
    except Exception as e:
        return json.dumps({"error": f"SerpAPI failed: {e}", "urls": [], "results": [], "count": 0})


In [4]:
from langchain.tools import tool

@tool("web_search_urls_fallback")
def web_search_urls_fallback(query: str, max_results: int = 8) -> str:
    """
    Intenta Tavily -> OpenAI Web Search -> Google (CSE o SerpAPI),
    y retorna URLs en JSON.
    """
    # 1) Tavily
    t = json.loads(tavily_search_urls.invoke({"query": query, "max_results": max_results}))
    if t.get("urls"):
        t["provider"] = "tavily"
        return json.dumps(t, ensure_ascii=False)

    # 2) OpenAI web search
    o = json.loads(openai_websearch_urls.invoke({"query": query, "max_results": max_results}))
    if o.get("urls"):
        o["provider"] = "openai_websearch"
        return json.dumps(o, ensure_ascii=False)

    # 3) Google (elige UNA de estas dos)
    g = json.loads(google_cse_search_urls.invoke({"query": query, "max_results": max_results}))
    if g.get("urls"):
        g["provider"] = "google_cse"
        return json.dumps(g, ensure_ascii=False)

    # Si usas SerpAPI, cambia lo anterior por:
    # g = json.loads(serpapi_search_urls.invoke({"query": query, "max_results": max_results}))
    # if g.get("urls"):
    #     g["provider"] = "serpapi"
    #     return json.dumps(g, ensure_ascii=False)

    # Nada funcionó
    return json.dumps({
        "provider": None,
        "urls": [],
        "results": [],
        "count": 0,
        "errors": {
            "tavily": t.get("error"),
            "openai_websearch": o.get("error"),
            "google": g.get("error"),
        }
    }, ensure_ascii=False)


In [5]:
from langchain_openai import ChatOpenAI
from langchain.agents import create_tool_calling_agent, AgentExecutor
from langchain_core.prompts import ChatPromptTemplate

llm = ChatOpenAI(model="gpt-4o-mini", temperature=0)

tools = [web_search_urls_fallback]  # el agente SOLO ve la tool orquestadora

prompt = ChatPromptTemplate.from_messages([
    ("system", "Eres un asistente que encuentra fuentes en la web. Cuando necesites buscar, usa la herramienta y devuelve URLs relevantes."),
    ("human", "{input}")
])

agent = create_tool_calling_agent(llm, tools, prompt)
executor = AgentExecutor(agent=agent, tools=tools, verbose=True)

resp = executor.invoke({
    "input": "Encuentra fuentes oficiales sobre requisitos de visado para colombianos viajando a Japón"
})

print(resp["output"])


ImportError: cannot import name 'create_tool_calling_agent' from 'langchain.agents' (c:\Users\000010478\Downloads\agentes_programas\.venv\Lib\site-packages\langchain\agents\__init__.py)

In [ ]:
prompt = ChatPromptTemplate.from_messages([
    ("system", """Devuelve SIEMPRE un JSON con:
{
 "question": "...",
 "urls": ["..."],
 "provider": "...",
 "notes": "..."
}
Si necesitas buscar, usa la herramienta."""),
    ("human", "{input}")
])